In [1]:
# !pip install -U -q kaggle --force

In [2]:
# from google.colab import files
# f=files.upload()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [5]:
# root_path = 'gdrive/My Drive/' 

In [6]:
# !mkdir -p ~/.kaggle

In [7]:
# !cp kaggle.json ~/.kaggle/

In [8]:
# !chmod 600 /root/.kaggle/kaggle.json

In [9]:
# !kaggle competitions download -c nnfl-lab-3-nlp

In [10]:
# %%bash
# cd /content
# unzip nnfl-lab-3-nlp.zip

In [11]:
import numpy as np 
import pandas as pd 
import os

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, Input, Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import re

from nltk.corpus import stopwords
import itertools

Using TensorFlow backend.


## Training Data

In [23]:
import nltk
nltk.download('stopwords')
stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harnoor_dhingra/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
df_original = pd.read_csv("nlp_train.csv", sep=",")

In [13]:
df = pd.DataFrame.copy(df_original)

In [14]:
df.head()

,offensive_language,tweet
0,3.0,Zuko gets bitches
1,1.0,Zimmerman we comin for yo life bitch. http://t...
2,3.0,Zhou Mi was just layin on his bed and SM just ...
3,3.0,Zelda bitches lol @joeylattime https://t.co/Cp...
4,0.0,Zack still questions my love for Oreos lol


In [16]:
# df['offensive_language'].value_counts().plot.bar()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13000 entries, 0 to 12999
Data columns (total 2 columns):
offensive_language    13000 non-null float64
tweet                 13000 non-null object
dtypes: float64(1), object(1)
memory usage: 203.2+ KB


## Testing Data

In [18]:
test_df = pd.read_csv("_nlp_test.csv", sep=",")

In [19]:
submission_df = test_df.copy()

In [20]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
tweet                 10000 non-null object
offensive_language    10000 non-null int64
dtypes: int64(1), object(1)
memory usage: 156.3+ KB


In [21]:
test_df = test_df.drop(['offensive_language'], axis = 1)

In [22]:
nb_samples = test_df.shape[0]
nb_samples

10000

In [24]:
# max_seq_length = max(df.tweet.map(lambda x: len(x)).max(),
#                      test_df.tweet.map(lambda x: len(x)).max())
max_seq_length = 100





## Embedding Layer

In [25]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']

# Iterate over the questions only of both training and test datasets
for dataset in [df,test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for sentence in ['tweet']:

            s2n = []  # q2n -> sentence numbers representation
            for word in text_to_word_list(row[sentence]):

                # Check for unwanted words
                if word in stops:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    s2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    s2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.at[index, sentence] = s2n

In [26]:
embeddings_index = {}
f = open('glove.twitter.27B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [27]:
EMBEDDING_DIM = 100
embeddings = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embeddings[i] = embedding_vector

In [28]:
from keras.layers import Embedding

embedding_layer = Embedding(len(embeddings),
                            EMBEDDING_DIM,
                            weights=[embeddings],
                            input_length=max_seq_length,
                            trainable=False)

In [29]:
from keras.preprocessing.sequence import pad_sequences
max_length_of_text = 100

X = pad_sequences(df['tweet'], maxlen=max_seq_length)
X_test = pad_sequences(test_df['tweet'], maxlen=max_seq_length)

In [30]:
y = df['offensive_language']
X_train, X_validation, y_train, y_validation = train_test_split(X,y, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_validation.shape,y_validation.shape)

(10400, 100) (10400,)
(2600, 100) (2600,)


In [31]:
lstm_out = 128
length_of_text = 100
batch_size = 32
inputs = Input(shape=(max_seq_length,), dtype='int32')
x = embedding_layer(inputs)
x = Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))(x)
x = Dense(1,activation='linear')(x)
model = Model(inputs, x)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          3503500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 3,738,253
Trainable params: 234,753
Non-trainable params: 3,503,500
_________________________________________________________________
None


In [32]:
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['mean_squared_error']);
model.compile(loss='mean_squared_error',optimizer='adam');

In [33]:
# es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

In [34]:
# model.fit(X_train, y_train, batch_size = batch_size, epochs = 10, callbacks=[es])

In [35]:
model.load_weights('2017A7PS0081G.h5')

In [36]:
score = model.evaluate(X_validation, y_validation, batch_size = batch_size)
print("Validation Loss: %.2f" % (score))

2600/2600 [==============================] - 3s 977us/step
Validation Loss: 0.33


In [0]:
# model.save_weights("model_try.h5")

## Predict

In [37]:
# predict = model.predict(X_test, steps=np.ceil(nb_samples/batch_size))
predict = model.predict(X_test)

In [38]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
tweet    10000 non-null object
dtypes: object(1)
memory usage: 78.2+ KB


In [39]:
submission_df['offensive_language'] = predict
# test_df['offensive_language'] = np.argmax(predict, axis=-1)

In [40]:
submission_df.head()

,tweet,offensive_language
0,I dnt let NOBODY borrow my shit&amp; i let My ...,2.842477
1,"I didnt buy any alcohol this weekend, and only...",1.925456
2,I didn&#8217;t go to school here RT @LmaoAtDos...,2.779803
3,I didn't wanna see your hoe ass pictures anywa...,2.773556
4,I didn't know my nig G-Eazy had reached a mill...,1.868626


In [41]:
# test_df['offensive_language'].value_counts().plot.bar()

## Submission CSV

In [42]:
# submission_df = test_df.copy()

In [43]:
submission_df.to_csv('submission_try.csv', index=False)

In [44]:
submission_df.head()

,tweet,offensive_language
0,I dnt let NOBODY borrow my shit&amp; i let My ...,2.842477
1,"I didnt buy any alcohol this weekend, and only...",1.925456
2,I didn&#8217;t go to school here RT @LmaoAtDos...,2.779803
3,I didn't wanna see your hoe ass pictures anywa...,2.773556
4,I didn't know my nig G-Eazy had reached a mill...,1.868626


In [0]:
# from google.colab import files
# files.download("../content/model_try.h5")
# files.download("../content/submission_try.csv")

In [45]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
    csv = submission_df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
create_download_link(df)